## Backtesting Framework

In [1]:
import pandas as pd

In [18]:
df = pd.read_csv("Data/TrainingData.csv")

In [19]:
df = df.drop(columns=["Unnamed: 0", 'Unnamed: 0.1']).rename(columns={'instrumnet': 'instrument'})

In [20]:
df.head()

,date,open,high,low,close,volume,instrument
0,2012-01-02,87.95,93.80,87.10,89.15,124736,A2ZINFRA
1,2012-01-03,89.55,91.15,89.10,89.70,275034,A2ZINFRA
2,2012-01-04,90.45,90.50,87.50,88.35,241157,A2ZINFRA
3,2012-01-05,89.10,93.45,86.55,87.60,226102,A2ZINFRA
4,2012-01-06,87.00,89.45,86.30,87.70,97276,A2ZINFRA


In [21]:
stock_instruments = df['instrument'].unique()

In [ ]:
def get_trade_type():
    #logic to weather buy, sell or hold
    pass
def register_trade():
    pass


In [25]:
# %% [markdown]
# # Backtest Your Strategy

# %%
import pandas as pd
from datetime import datetime
from TradeReplay.decision  import DecisionStrategy
from TradeReplay.session   import Session

# %% define a simple momentum strategy
class MomentumStrategy(DecisionStrategy):
    def __init__(self, lookback=3):
        self.lookback = lookback
        self.history  = {}

    def decide(self, session, daily_df):
        """
        daily_df: pd.DataFrame with columns [date, open, high, low, close, volume, instrument]
        session: gives you session.portfolio and session.tradebook
        """
        for sym, group in daily_df.groupby("instrumnet"):
            closes = group["close"].tolist()
            # store history
            self.history.setdefault(sym, []).extend(closes)
            # only act if we have lookback+1 days
            prices = self.history[sym]
            if len(prices) >= self.lookback + 1:
                # simple momentum: if today’s close > close N days ago → BUY, else SELL
                if prices[-1] > prices[-1-self.lookback]:
                    session.portfolio.buy(sym, prices[-1], group["date"].iloc[0], quantity=1)
                elif sym in session.portfolio.holdings:
                    session.portfolio.sell(sym, prices[-1], group["date"].iloc[0], quantity=1)

# %% parameters
DATA_PATH  = "Data/TrainingData.csv"
START_DATE = datetime(2012,1,2)
END_DATE   = datetime(2012,6,30)

# %% run backtest
sess = Session(
    data_path = DATA_PATH,
    start_date=START_DATE,
    end_date  = END_DATE,
    brokerage = 0.0005,
    investment=100_000
)
strategy = MomentumStrategy(lookback=5)
sess.run(strategy)

# %% inspect results
trades_df = pd.read_csv(sess.tradebook.filepath)
display(trades_df.head(), trades_df.tail())
print("Final portfolio value:",
      sess.portfolio.total_value(
          # use last known prices for mark-to-market
          { row.instrument: row.close
            for _, row in sess.loader.df[sess.loader.df.date == END_DATE].iterrows() }
      )
)

,symbol,action,date,price,quantity,net_amount
0,FEDERALBNK,BUY,2012-01-07 00:00:00,180.65,1,196.670325
1,HINDZINC,BUY,2012-01-07 00:00:00,123.15,1,139.141575
2,RELIANCE,BUY,2012-01-07 00:00:00,357.38,1,373.488690
3,TECHM,BUY,2012-01-07 00:00:00,149.88,1,165.884940
4,A2ZINFRA,BUY,2012-01-09 00:00:00,89.50,1,105.474750


,symbol,action,date,price,quantity,net_amount
3392,IZMO,SELL,2012-04-11 00:00:00,18.50,1,2.560750
3393,ITI,SELL,2012-04-13 00:00:00,24.25,1,8.307875
3394,KOTHARIPET,BUY,2012-04-13 00:00:00,5.70,1,21.632850
3395,KOTHARIPET,SELL,2012-04-17 00:00:00,5.70,1,-10.232850
3396,DCW,SELL,2012-04-23 00:00:00,11.75,1,-4.185875


Final portfolio value: -14.244225000053738
